In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandana as pdna
import geopandas as gpd
import pandas as pd
import osmnx as ox
import networkx as nx
import sys
#import matplotlib.pyplot as plt


C:\Users\z3258367\AppData\Local\Temp\ipykernel_45628\2874976097.py:3: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


In [4]:
# adding functions 
sys.path.insert(0, '/home/vagvaf/projects/THERE')
sys.path.insert(0, 'C:\\Users\\z3258367\\OneDrive - UNSW\\#PhD\\Walkability\\Other Cities\\Open-Walk-Index')
from walkability_functions import *

## Config variables 

before proceeding set up various config variables

In [30]:
# working folder
folder = "/home/vagvaf/projects/THERE/#Example Notebooks/Goteborg"

#choose a projected CRS to be used for all distance calculations.
proj_crs = "EPSG:3006"

#study area boundaries
study_area = 'Goteborg_metropolitan_dissolve.gpkg'

#poi dictionary

poi_dictionary = {
    'grocery':{
        'category':['grocery']
    }
}

#network related configs
maximum_dist=2400

#results
output_file = 'Goteborg_results_walk_230723.gpkg'

In [7]:
# based on https://commons.wikimedia.org/wiki/File:Metropolitan_Gothenburg_ver.3.PNG
place_gdf = gpd.read_file(study_area, crs="EPSG:4326").to_crs(proj_crs)

In [ ]:
# check that we have the right area
#place_gdf.geometry.plot()

## Import Data

Data sources:
1. Grocery stores data

### Add grocery store data

In [60]:
# import grocery_stores.gpkg from Data/Sweden
#grocery_stores = gpd.read_file('grocery_stores.gpkg').to_crs(proj_crs)
grocery_stores = gpd.read_file('C:\\Users\\z3258367\\OneDrive - UNSW\\#PhD\\Data\\Sweden\\grocery_stores.gpkg').to_crs(proj_crs)

In [61]:
# add category column, all values 'grocery' to grocery_stores
grocery_stores['category'] = 'grocery'                                                                                     

In [62]:
#pois = pd.concat([osm_pois, employment_centrs])
pois = grocery_stores

pois = gpd.clip(pois, place_gdf.to_crs(proj_crs))

### Categorise and weight POIs

Choose walk index weightings, and output the sums of each category and the total to check. The walk index will be out of 100 regardless of this sum, but it is important to note that eg. shopping is only '10% of the walk index' if shopping is 10 out of 100.

In [40]:
poi_parameters = pd.read_csv(("poi_parameters.csv"), index_col=0)

### Import network

Pandana expects edges to have a two item index based on the same IDs as the node index. (with thanks to https://github.com/shriv/accessibility-series/blob/master/Accounting%20for%20hills%20in%20accessibility%20analyses.ipynb)

In [ ]:
#G = ox.graph_from_place(place, network_type='walk')
G = ox.graph.graph_from_polygon(place_gdf.to_crs('EPSG:4326').geometry[0], network_type='walk')

In [14]:
#ox.io.save_graphml(G, filepath=None, gephi=False, encoding='utf-8')
G = ox.io.load_graphml(filepath="Goteborg walk network", node_dtypes=None, edge_dtypes=None, graph_dtypes=None)

In [15]:
# Get nodes and edges as geodataframes (gdfs) from OSMNX network
graph_df = ox.graph_to_gdfs(G)
nodes_gdfs = graph_df[0].to_crs(proj_crs)
edges_gdfs = graph_df[1].to_crs(proj_crs)

## Pandana network creation.

In [16]:
# with new OSMnx graph from polygon seems to be different
edges_gdfs = edges_gdfs.reset_index()
# Setting indices of Edges gdfs to match expected dataframe for Pandana
edges_gdfs['from_idx'] = edges_gdfs['u']
edges_gdfs['to_idx'] = edges_gdfs['v']
edges_gdfs= edges_gdfs.set_index(['from_idx', 'to_idx'])
edges_gdfs.index.names= ['','']

# Setting indices of Nodes gdfs to match expected dataframe for Pandana
nodes_gdfs.index.name = 'id'
# Create a pandana network with data extracted from an OSMNX graph
distance_network = pdna.Network(nodes_gdfs.geometry.x, nodes_gdfs.geometry.y,
                                   edges_gdfs['u'], edges_gdfs['v'], 
                                   edges_gdfs[['length']])

### Pandana network querying. 

### Testing different parameters

In [54]:
test_parameters = poi_parameters.loc[['grocery']]

In [57]:
poi_weights = test_parameters['weight'].copy()
poi_lambdas = test_parameters['diminishing_returns_constant'].copy()
poi_variables = test_parameters['variable'].copy()
poi_nums = test_parameters['num_pois'].copy()
poi_gammas = test_parameters['distance_constant'].copy()

In [58]:
poi_parameters['num_pois'] = 50
poi_nums = poi_parameters['num_pois'].copy()

In [25]:
# for perceived access to grocery store testing
distance_constants = [0.0005, 0.00075, 0.001, 0.00125, 0.0015]
diminishing_returns_constants = [0.1, 0.2, 0.3, 0.5, 0.7]

In [73]:
i = 0
for dist_const in distance_constants:
    for dim_const in diminishing_returns_constants:
        poi_gammas.loc['grocery'] = dist_const
        poi_lambdas.loc['grocery'] = dim_const
        
        results_walk = there_index(distance_network, grocery_stores, poi_dictionary, poi_weights, poi_gammas,
                            poi_nums, poi_lambdas, poi_variables, distance=maximum_dist)

        small_results = result_rounding(results_walk)

        #output_file = 'Goteborg_results_walk_300723' + '_dist' + str(dist_const) + '_dim' + str(dim_const) + '.gpkg'
        #small_results.to_file(output_file, driver="GPKG")

        if i == 0:
            results = gpd.GeoDataFrame(small_results, geometry = gpd.GeoSeries.from_xy(results_walk.x, results_walk.y, crs=proj_crs))
            results.drop(columns=['THERE_Index'], inplace=True)

        col_name = 'THERE_dist_' + str(dist_const) + '_dim_' + str(dim_const)   
        results.insert(results.shape[1], col_name, small_results['THERE_Index'])

        i += 1
        print(dist_const, dim_const, "mean:", np.mean(results_walk['THERE_Index']))

results.to_csv('Goteborg_walk_25_grocery_300723.csv')

Finished category: grocery
Maximum score: 68.17072578875297 out of 100
0.0005 0.1 mean: 13.057542021214275
Finished category: grocery
Maximum score: 76.89436847101179 out of 100
0.0005 0.2 mean: 17.956343799175094
Finished category: grocery
Maximum score: 72.97092241434756 out of 100
0.0005 0.3 mean: 19.850447928079607
Finished category: grocery
Maximum score: 60.597757534248565 out of 100
0.0005 0.5 mean: 19.949773742287157
Finished category: grocery
Maximum score: 49.65577673420598 out of 100
0.0005 0.7 mean: 18.171039237340594
Finished category: grocery
Maximum score: 68.17072578875297 out of 100
0.00075 0.1 mean: 9.81015901201347
Finished category: grocery
Maximum score: 76.89436847101179 out of 100
0.00075 0.2 mean: 13.788498306184767
Finished category: grocery
Maximum score: 72.97092241434756 out of 100
0.00075 0.3 mean: 15.466822283327701
Finished category: grocery
Maximum score: 60.597757534248565 out of 100
0.00075 0.5 mean: 15.839730848732778
Finished category: grocery
Maximu

Note that the mean below is based on every network point, in the case of Greater Gothenburg this includes a lot of empty areas. To get a better indicator of average access for the population, need to produce population-weighted results as in section below.

In [22]:
max(results_walk['THERE_Index']), np.mean(results_walk['THERE_Index'])

(72.97092241434756, 12.276192236634376)

The current approach is to find up to x closest nodes for each category within the maximum distance. Then look up the attractiveness at each one (just count for most), apply a distance decay function to each distance, apply diminishing returns to the resulting total opportunity, and sum.

An alternative approach which would be more convenient would be to use the Pandana 'aggregate' function which aggregates from all nodes within the maximum distance including applying a decay function. However, there is limited ability to change the distance decay rate within the aggregation function. It can either be flat (no decay), linear (going to 0 at the max distance), or exponential where beta is set as 1/max distance. For walking I would like a beta of 0.001, but this requires the radius to be 1000m. If the radius is 2400m, beta is only 0.0004. This can be changed in the future if the Pandana function is updated to take a decay parameter.

## Export results

Reduce the number of decimal places before export, distances to destinations do not need to be below 1m.

In [26]:
def result_rounding(results):
    # reduces results size for export
    # score columns such as THERE_Index, employment_subtotal etc -> 3 decimal places
    # distance columns such as employment1 -> 0 decimal places (nearest metre)
    # avoid doing anything to connect_id, x or y
    rounding_dict = {**{k:3 for k in results.columns if "Index" in k or "." in k
                        and 'connect_id' not in k},
                     **{k:0 for k in results.columns if "Index" not in k and "." not in k
                        and k != 'x'
                        and k != 'y'}}
    return results.round(rounding_dict)

In [24]:
small_results = gpd.GeoDataFrame(result_rounding(results_walk), geometry = gpd.GeoSeries.from_xy(results_walk.x, results_walk.y, crs=proj_crs))

In [25]:
small_results.to_file(output_file)